In [14]:
from bardapi import Bard

token = 'WwhtZr3uasaGA9cyQATKdVWr7TBe0IeBhYZLJigB2svu9J_TwKpjYBTalu85cdGWPkVIpA.'
bard = Bard(token=token)

CONTEXT = "Programmable devices have existed for centuries. As early as the 9th century, a programmable music sequencer was invented by the Persian Banu Musa brothers, who described an automated mechanical flute player in the Book of Ingenious Devices.[4][5] In 1206, the Arab engineer Al-Jazari invented a programmable drum machine where a musical mechanical automaton could be made to play different rhythms and drum patterns, via pegs and cams.[6][7] In 1801, the Jacquard loom could produce entirely different weaves by changing the \"program\" – a series of pasteboard cards with holes punched in them. Code-breaking algorithms have also existed for centuries. In the 9th century, the Arab mathematician Al-Kindi described a cryptographic algorithm for deciphering encrypted code, in A Manuscript on Deciphering Cryptographic Messages. He gave the first description of cryptanalysis by frequency analysis, the earliest code-breaking algorithm. The first computer program is generally dated to 1843, when mathematician Ada Lovelace published an algorithm to calculate a sequence of Bernoulli numbers, intended to be carried out by Charles Babbage's Analytical Engine. Data and instructions were once stored on external punched cards, which were kept in order and arranged in program decks. In the 1880s, Herman Hollerith invented the concept of storing data in machine-readable form.[10] Later a control panel (plug board) added to his 1906 Type I Tabulator allowed it to be programmed for different jobs, and by the late 1940s, unit record equipment such as the IBM 602 and IBM 604, were programmed by control panels in a similar way, as were the first electronic computers. However, with the concept of the stored-program computer introduced in 1949, both programs and data were stored and manipulated in the same way in computer memory.[11]"
bard.get_answer(f"When was the first computer program made? Answer using only this paragraph: {CONTEXT}. Don't infer things in your answer.")['content']

"According to the paragraph, the first computer program was made in 1843 by Ada Lovelace. She published an algorithm to calculate a sequence of Bernoulli numbers, intended to be carried out by Charles Babbage's Analytical Engine."

In [3]:
import os
from google.cloud import aiplatform

PROJECT_ID = "moonlit-palace-387116"
REGION = "europe-west4"  # @param {type: "string"}
CREDS = "??"
KEY_NAME = "??"

from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

MODEL_ARTIFACT_DIR = "triton-pytorch"  # @param {type:"string"}
REPOSITORY = "custom-container-prediction-sdk"  # @param {type:"string"}
IMAGE = "triton-pytorch"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "triton-pytorch"  # @param {type:"string"}
LOCAL_MODEL_ARTIFACTS_DIR = "model_artifacts"  # @param {type:"string"}


aiplatform.init(
    # your Google Cloud Project ID or number
    # environment default used is not set
    project=PROJECT_ID,

    # the Vertex AI region you will use
    # defaults to us-central1
    location=REGION,

    # Google Cloud Storage bucket in same region as location
    # used to stage artifacts
    staging_bucket='gs://my_staging_bucket',

    # custom google.auth.credentials.Credentials
    # environment default creds used if not set
    credentials=CREDS,

    # customer managed encryption key resource name
    # will be applied to all Vertex AI resources if set
    encryption_spec_key_name=KEY_NAME,

    # the name of the experiment to use to track
    # logged metrics and parameters
    experiment='chatbot',

    # description of the experiment above
    experiment_description='Chatbot'
)

In [ ]:
! gsutil mb -l PROJECT_ID $BUCKET_URI

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "EleutherAI/gpt-j-6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

C:\Users\point\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\point\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)



KeyboardInterrupt



In [ ]:
def get_answer(query):
    inputs = tokenizer.encode("Q: " + query + "\nA:", return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_length=1024, do_sample=True)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = answer.split("A:")[1].strip()
    return answer